# SpaCy Named Entity Recognition Implementation

## (AT/TA skip) Mount drive with data 

In [88]:
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [89]:
#AT/TA skip
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
#AT/TA skip
%cd  /content/drive/Shared\ drives/256/ASKus

/content/drive/Shared drives/256/ASKus


## Data Exploration and Preprocessing

### Load train.csv containing matrix with paper id's corresponding to dataset label

In [91]:
import pandas as pd

train = pd.read_csv('./data/train.csv')
train

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
...,...,...,...,...,...
19656,b3498176-8832-4033-aea6-b5ea85ea04c4,RSNA International Trends: A Global Perspectiv...,RSNA International COVID-19 Open Radiology Dat...,RSNA International COVID Open Radiology Database,rsna international covid open radiology database
19657,f77eb51f-c3ac-420b-9586-cb187849c321,MCCS: a novel recognition pattern-based method...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...
19658,ab59bcdd-7b7c-4107-93f5-0ccaf749236c,Quantitative Structure–Activity Relationship M...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...
19659,fd23e7e0-a5d2-4f98-992d-9209c85153bb,A ligand-based computational drug repurposing ...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...


In [ ]:
##script used to take json text files and train.csv and convert them to SpaCy friendly format (takes approx. 80 minutes to complete)

##create array to store tuples of publication text and entity (dataset name) locations (this is how SpaCy wants it)
# SPACY_TRAIN = []

##counter used to track progress
# count = 0

##loop through train.csv rows
# for index, row in train.iterrows():
#   print(count)
#   this_pub = ""

##load json file corresponding to publication id in the train.csv row, store known dataset label
#   this_pub_id = './data/train/' + row['Id'] + '.json'
#   this_data_lab = row['dataset_label']
#   this_pub_json = json.load(open(this_pub_id))

##append all text from the publication in to one (super-long!) string
#   for section in this_pub_json:
#     this_pub += section['text']

##Find locations of dataset in publications and store in an entity dictionary
#   this_ent_dict = {"entities": []}
#   for m in re.finditer(this_data_lab, this_pub):
#     this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))

## append the tuple of the entire publication text and the entity locations in the text
#   SPACY_TRAIN.append((this_pub, this_ent_dict))
#   count+=1

# SPACY_TRAIN

In [ ]:
#write SpaCy friendly training data into a text file

# file = open("./data/spacy_train.txt", "w")
# SPACY_TRAIN_File = repr(SPACY_TRAIN)
# file.write(SPACY_TRAIN_File)
# file.close()

In [110]:
#script to read in SpaCy friendly data from text file (script modified from one here: https://www.geeksforgeeks.org/how-to-read-large-text-files-in-python/)
import fileinput
import time
  
#time at the start of program is noted
start = time.time()
spacy_train = ""
#keeps a track of number of lines in the file
count = 0
for lines in fileinput.input(["./data/spacy_train.txt"]):
    spacy_train = lines
    count = count + 1
      
#time at the end of program execution is noted
end = time.time()
  
#total time taken to print the file
print("Execution time in seconds: ",(end - start))
print("No. of lines printed: ",count)

#number of chars in file (exceeds one billion)
print("Number of characters in the data file (too many): ", len(spacy_train))

Execution time in seconds:  10.072824001312256
No. of lines printed:  1
Number of characters in the data file (too many):  1093591763


In [111]:
#SpaCy friendly training data was read in from .txt file as a string - use following ton convert back to list of tuples
#Source code: https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
import ast

DATA = ast.literal_eval(spacy_train)

print("number of (paper, dataset_location) tuples in the data: ", len(DATA))

number of (paper, dataset_location) tuples in the data:  19661


In [120]:
#split data into 70% training and 30% test
#Train only on the 70% and keep the rest for evaluation purposes

import random

#randomly shuffle the tuples
random.shuffle(DATA)

train_mark = (len(DATA)*7//10)
TRAIN_FULL = DATA[:train_mark]
TEST_FULL = DATA[train_mark:]

In [121]:
#Upon attempting training, there was not enough RAM/it was too slow to process all texts in their entirities.
#As such, following script is used to compress the training data.  It breaks the publications up into sentences.  It then picks,
# from each publication, one sentence that contains the data set name inside it, and one sentence that does not.
import re

#set our custom entity label to be trained on later
LABEL = 'data'

TRAIN_DATA = []

#loop through tuples in original data
#hit_count and miss_count serve as counters for how many sentences we have already extracted from this particular paper
for paper in TRAIN_FULL:
  hit_count = 0
  miss_count = 0

  #determine if the paper we are looking at contains any entities
  entities = paper[1]['entities']
  if len(entities) > 0:
    #store the dataset name for this publication
    dataset = paper[0][entities[0][0]:entities[0][1]-1]

  #split data into sentences, approximately (based on the '.' puncutation mark)
  sentences = paper[0].split('.')

  #loop through sentences
  for sentence in sentences:
    this_ent_dict = {"entities": []}
    if dataset in sentence:
      hit_count +=1

      #find all occurences of explicit mentions of the dataset in this sentence
      for m in re.finditer(dataset,sentence):
        this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))
      #compressed data: append a tuple of just the sentence (as opposed to the entire paper) containing the dataset entity
      TRAIN_DATA.append((sentence, this_ent_dict))
    
    #after find one sentence containing a dataset, append two that do not contain it
    elif hit_count > 0 and miss_count <= hit_count:
      miss_count += 1
      TRAIN_DATA.append((sentence, this_ent_dict))

    #when have two sentences from this paper (one containing dataset and one without), move on to next paper
    if hit_count >= 1 and miss_count >= 2:
      break

In [122]:
#number of entries ((sentence, entity-locations) tuples) in the compressed dataset
print("number of (sentence, datset_location) tuples in the compressed data: ", len(TRAIN_DATA))

number of (sentence, datset_location) tuples in the compressed data:  34230


In [123]:
#find and remove all sentences containing no chars (ex: "", " ", "      ") as these cause an error in the SpaCy training algorithm

count = 0
sen = 0
while sen < len(TRAIN_DATA):
  #if the sentence contains some chars, move on to the next sentence
  if TRAIN_DATA[sen][0].strip():
      sen += 1
  
  #if it is a blank sentence, delete it
  else:
      # print(TRAIN_DATA[sen])
      del TRAIN_DATA[sen]
      count+=1


print("Number of blank sentences we removed: ", count)

Number of blank sentences we removed:  72


In [107]:
#split data into 70% training and 30% test
#Train only on the 70% and keep the rest for evaluation purposes

# import random

# #randomly shuffle the tuples
# random.shuffle(new_spacy)

# train_mark = (len(new_spacy)*7//10)
# TRAIN_DATA = new_spacy[:train_mark]
# TEST_DATA = new_spacy[train_mark:]

## Training SpaCy NER with Custom Entities

In [ ]:
#script derived from https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

import spacy as sp
from spacy.util import minibatch, compounding
from pathlib import Path


#create empty model
nlp = sp.blank('en')
print("created blank english model")

#add ner (entity recognizer) to the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

#add our custom entity label to the entity recognizer
ner.add_label(LABEL)
optimizer = nlp.begin_training()

#set number of epochs we want our model to perform
n_iter = 3

#make sure we only train our ner pipe to speed up processing
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        print(itn)
        random.shuffle(TRAIN_DATA)
        losses = {}
        #split data into batches of 500 sentences
        batches = minibatch(TRAIN_DATA, size=500)
        for batch in batches:
            print('batch size', len(batch))
            #extract the sentence and the entities it contains for each sentence
            texts, annotations = zip(*batch)
            #update the weights
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                        losses=losses)
        #loss after this epoch
        print('Losses', losses)


# Save model 
output_dir = Path('./models/spacy')
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'SpaCy'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

created blank english model
0
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size 500
batch size

## Internal Evaluation

In [106]:
print("Loading from", output_dir)
nlp2 = sp.load(output_dir)

ground_truth = []
predictions = []
for test_text in TEST_FULL:
    this_gt = []
    for gt_ent in test_text[1]['entities']:
        dataset = test_text[0][gt_ent[0]:gt_ent[1]-1]
        if dataset not in this_gt:
            this_gt.append(dataset)
    ground_truth.append(this_gt)
    
    this_pred = []
    sentences = test_text[0].split('.')
    for sentence in sentences:
        doc = nlp2(sentence)
        for ent in doc.ents:
            print(ent.label_, ent.text)
            if len(ent.text) > 0 and ent.text not in this_pred:
                this_pred.append(ent.text)
    predictions.append(this_pred)
    
print(len(ground_truth))
print(len(predictions))

In [ ]:
for num in range(0, len(ground_truth)):

In [ ]:
import json

test = json.load(open('./data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json'))

this_pub = ""
for section in test:
  this_pub += section['text']

sentences = this_pub.split('.')
for sentence in sentences:
  doc = nlp(sentence)
  # print("Entities in '%s'" % test_text)
  if len(doc.ents) > 0:
    for ent in doc.ents:
        print(ent.label_, ent.text)


# count = 0
# for index, row in train.iterrows():
#   print(count)
#   this_pub = ""
#   this_pub_id = './data/train/' + row['Id'] + '.json'
#   this_data_lab = row['dataset_label']
#   this_pub_json = json.load(open(this_pub_id))
#   for section in this_pub_json:
#     this_pub += section['text']
#   this_ent_dict = {"entities": []}
#   for m in re.finditer(this_data_lab, this_pub):
#     this_ent_dict['entities'].append((m.start(), m.end()+1, LABEL))
#   SPACY_TRAIN.append((this_pub, this_ent_dict))
#   count+=1

# SPACY_TRAIN

data Rural-Urban Continuum Codes (


In [ ]:
this_pub = ""
for section in data:
  this_pub += section['text']


In [ ]:
import re
this_data_lab = "National Education Longitudinal Study"
for m in re.finditer(this_data_lab, this_pub):
  this_pub[m.start():m.end()+1]
print(this_pub[2937:2975])
#[0, 5, 10, 15]

ely to attain a college degree than co


In [ ]:
# !pip install -U spacy
# !python -m spacy download en

#check num json files in train (should be 14,316)
# import glob
# len(glob.glob1("./data/train", "*.json"))